In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('E:/Jupyter Notebook/인공지능인문학/텍스트 윤리검증 데이터/discrimination_data.csv')
df

,Unnamed: 0,text,types,intensity
0,0,남자들은 도대체 왜 그러는걸까?,DISCRIMINATION,1.0
1,1,말하면서 침 개많이 튀길것 같아.,DISCRIMINATION,3.0
2,2,머리가 멍청해서 못읽는거아니냐?,DISCRIMINATION,3.0
3,3,"동유럽, 러시아 애들은 가난해도 사랑으로 결혼하는데 헬조선은 자본주의의 지옥 그 자체다",DISCRIMINATION,1.0
4,4,원래 남자는 365일 발정기야,DISCRIMINATION,2.0
...,...,...,...,...
4430,4430,조상님한테 제사도 안 지내고. 넌 복 받을 게 없겠다.,DISCRIMINATION,1.0
4431,4431,여자 성격이 그렇게 깐깐해서 시집은 가겠냐? ㅋㅋㅋ,DISCRIMINATION,1.0
4432,4432,이슬람 애들 한 번 챙겨줬다가 유럽 꼴 날까 봐 무섭다,DISCRIMINATION,1.0
4433,4433,나도. 이슬람 애들은 왜 그렇게 기를 쓰고 남의 나라 차지하려는지 모르겠어,DISCRIMINATION,1.0


In [3]:
# df = df['text']
# df.to_csv('discrimination_data.csv', index=False)

# Tokenization (한글 형태소 단위로 쪼개기)

In [4]:
import konlpy
import re

# tokenization 함수를 만들어둠
def tokenize_korean_text(text):
    # text = re.sub(r'[^,.?!\w\s]','', text)  
    ## ,.?!와 문자+숫자+_(\w)와 공백(\s)만 남김  # 앞에 r을 붙여주면 deprecation warning이 안뜸 (raw string으로 declare)
    
    okt = konlpy.tag.Okt()
    Okt_morphs = okt.pos(text)   # stem=True로 설정하면 동사원형으로 바꿔서 return
    
    words = []
    for word, pos in Okt_morphs:
        if  pos == 'Noun':  # 이 경우에는 명사만 남김
            words.append(word)

    words_str = ' '.join(words)
    return words_str


# df['text']를 하나씩 tokenize해서 list로 저장
tokenized_list = []

for text in df['text']:
    tokenized_list.append(tokenize_korean_text(text))

print(len(tokenized_list))
print(tokenized_list[0])

4435
남자 도대체 왜


# Vectorization & LDA

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

## vector화

In [6]:
#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 10% 이상으로 자주 등장하는 단어는 제외
    # bigram도 포함

feat_vect = count_vectorizer.fit_transform(tokenized_list)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (4435, 1000)


## 토픽모델링: LDA

In [7]:
lda = LatentDirichletAllocation(n_components=8)  # 토픽 수는 8개로 설정
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=8)

## 토픽별 연관어 출력

In [8]:
def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]  # argsort는 array의 내림차순 정렬
        top_indexes=topic_word_indexes[:num_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

Topic # 0
남자 인간 장애인 한국 집단 인간 집단 인간 인간 인물 여자 남자 남자 여자
Topic # 1
사람 냄새 출신 수준 부모 누가 동남아 무슨 대학 친구
Topic # 2
진짜 흑인 결혼 짱깨 정도 자식 어차피 여자 결혼 게임 수저
Topic # 3
생각 전라도 요즘 문제 사람 이유 서울 운동 외모 운전
Topic # 4
주제 한남 사람 어디 무슨 학교 무조건 저런 시골 정신
Topic # 5
머리 엄마 때문 보고 우리나라 정말 절대 여자 여자 차별 성격
Topic # 6
그냥 남자 얼굴 게이 우리 여자애 나라 원래 페미 군대
Topic # 7
조선족 하나 역시 거지 공부 중국 나이 그게 보기 동네


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# LDA 시각화: pyLDAvis

In [9]:
!pip install pyLDAvis

In [10]:
import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

C:\Users\user\anaconda3\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


- (참고): λ 를 1 로 설정하면 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미이며, λ를 0 에 가깝게 설정할수록 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미이다.
- 'Intertopic Distance Map'을 통해 토픽 개수는 4개가 적당함을 알 수 있다.

# 문서별 토픽 할당
## 각 문서별로 가장 가까운 topic으로 할당

In [11]:
# 문서별로, 가장 확률이 높은 topic으로 할당해줌

doc_topic = lda.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    topic_pr = doc_topic[n].max()
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

doc_topic_df.head()

,Doc_Num,Topic,Percentage
0,0,6,0.708171
1,1,0,0.125000
2,2,5,0.562500
3,3,1,0.687386
4,4,6,0.781162


In [12]:
doc_topic_df = doc_topic_df.join(df)
doc_topic_df.head()

,Doc_Num,Topic,Percentage,Unnamed: 0,text,types,intensity
0,0,6,0.708171,0,남자들은 도대체 왜 그러는걸까?,DISCRIMINATION,1.0
1,1,0,0.125000,1,말하면서 침 개많이 튀길것 같아.,DISCRIMINATION,3.0
2,2,5,0.562500,2,머리가 멍청해서 못읽는거아니냐?,DISCRIMINATION,3.0
3,3,1,0.687386,3,"동유럽, 러시아 애들은 가난해도 사랑으로 결혼하는데 헬조선은 자본주의의 지옥 그 자체다",DISCRIMINATION,1.0
4,4,6,0.781162,4,원래 남자는 365일 발정기야,DISCRIMINATION,2.0


## 토픽별 문서 수 계산

In [13]:
doc_topic_df.groupby('Topic')[['Doc_Num']].count()

,Doc_Num
Topic,
0,976
1,504
2,541
3,483
4,419
5,430
6,559
7,523


## 토픽별로, 가장 높은 확률로 할당된 문서 top3 확인

In [14]:
for topic in range(len(doc_topic_df['Topic'].unique())):
    print('Topic #', topic, '-----------------------------')
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    print(top_pr_topics['text'].iloc[0])
    print(top_pr_topics['text'].iloc[1])
    print(top_pr_topics['text'].iloc[2], '\n')

Topic # 0 -----------------------------
그건 100프로 #@인간및인간집단.조직및단체.정당# 지지하는 경상도나 해당하는 얘기 아냐?
#@#에서 인구 많아지면 계속 #@인간및인간집단.조직및단체.정당# 지지할 애들이 늘어나는 거 아니냐?
#@인간및인간집단.인물.인물_무정보.여성#는 조국과 동급이네 

Topic # 1 -----------------------------
동남아는 항상 비가 와서 사람들도 냄새가 나나?
니도 그 나이대에 별반 차이없는 것 같은데? 부모한테 가정 교육 안 받은 티좀 내지마 저능아야
근데 조손가정 애들한테서는 특유의 냄새가 나지 않아? 흙냄새? 된장냄새? 그런거. 

Topic # 2 -----------------------------
야 여자는 임신을 하기 위해 태어난 몸인데 안돼 애기들한테 안좋아 난 진짜 싫어
뭐, 흑인 바비 인형 가지고 놀면서 인종에 대해 좀 배우라고 해.
원래 일본 연예인들은 관심 받으려고 위장 결혼 많이 함 

Topic # 3 -----------------------------
솔직히 외모도 센터에 실력도 이정도인 아이돌이 어딨냐? 잘못은 잘못이고 나같아도 안고 감
전라도 인이 서울 산다고 서울사람 돼냐?
애초에 여자가 차가지고 나와서 운전하는게 문제야 

Topic # 4 -----------------------------
그렇기는한데 얼마전 동남아 혼혈애가 전학을 왔는데 그런애는 우리애한테 피해만 줄거에요
돈이 필요해서 우리 아들 만나는 거 알아 돈줄테니까 우리 아들이랑 그만 만나
난 뚱뚱한 사람 보면 자기 관리도 제대로 못 하는 거 같아서 한심해 

Topic # 5 -----------------------------
인정, 너무 똑똑하고 되바라진 여자는 여자로서 가치가 없음.
역시 여자인생 최고 아웃풋은 취집이지 ㅋ
자고로 여자는 치마는 짧고 머리는 길어야지 

Topic # 6 -----------------------------
그래 우리는 헬조선에서 탈출해서

# 토픽의 개수를 바꿔가며 해보자

## 4개

In [15]:
lda = LatentDirichletAllocation(n_components=4)  # 토픽 수는 4개로 설정
lda.fit(feat_vect)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic # 0
진짜 거지 요즘 게이 주제 문제 생각 화장 저런 자기
Topic # 1
남자 무슨 장애인 그냥 조선족 하나 머리 여자애 원래 중국인
Topic # 2
남자 흑인 결혼 짱깨 얼굴 우리 나이 동네 여자 남자 엄마
Topic # 3
사람 인간 역시 나라 한국 집단 전라도 냄새 한남 인간 집단


C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [16]:
doc_topic = lda.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    topic_pr = doc_topic[n].max()
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

doc_topic_df = doc_topic_df.join(df)

for topic in range(len(doc_topic_df['Topic'].unique())):
    print('Topic #', topic, '-----------------------------')
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    print(top_pr_topics['text'].iloc[0])
    print(top_pr_topics['text'].iloc[1])
    print(top_pr_topics['text'].iloc[2], '\n')

Topic # 0 -----------------------------
뭐야 이 아저씨 대리기사 주제에 설치네?
넘사벽을 실제로 봐야 이 거지 주제를 제대로 파악하지.
대체 #@# 난민들을 왜 우리나라에 데려오는거냐 진짜 세금 아깝네 또 얼마나 잘해주는지 참. 

Topic # 1 -----------------------------
그냥 중국인 유전자 색출해서 세계적으로 중국인들은 공식적으로 인종차별해야 함.
장애인들 피해의식 개 심하지 않음? 무조건 양보 받으려고 함 ㅋ
여자가 시집을 왔으면 시댁부터 무조건 먼저 가야하는거 아니야? 

Topic # 2 -----------------------------
여자가 남자 능력 보고 결혼하지 그럼 뭐 보고 결혼해? 자원봉사냐?
솔직히 외모도 센터에 실력도 이정도인 아이돌이 어딨냐? 잘못은 잘못이고 나같아도 안고 감
군인 포상 휴가를 생리휴가랑 같다고 비교하고 있다 ㅋㅋ 저게 뭐임? 

Topic # 3 -----------------------------
음. 그런가 그래도 #@인간및인간집단.인물.정치인.남성# 대구 사람이잖아. 대구 사람들 다 꼰머
#@인간및인간집단.조직및단체.교육기관# 출신의 성공 비결은 오직 취집이지
결국 남은 건 막대한 손해에 아줌마 #@인간및인간집단.인물.인물_무정보.여성#, 한 물 간 걸그룹임 



## 5개

In [17]:
lda = LatentDirichletAllocation(n_components=5)
lda.fit(feat_vect)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic # 0
남자 장애인 생각 결혼 어디 전라도 공부 나라 여자 남자 출신
Topic # 1
남자 게이 짱깨 요즘 한남 하나 거지 머리 장애 보기
Topic # 2
사람 흑인 정도 화장 페미 학교 그게 때문 우리 인정
Topic # 3
인간 조선족 그냥 주제 우리 문제 인물 집단 얼굴 여성
Topic # 4
진짜 남자 무슨 원래 역시 여자애 냄새 애가 무조건 동남아


C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


## 6개

In [26]:
lda = LatentDirichletAllocation(n_components=6)
lda.fit(feat_vect)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic # 0
진짜 흑인 얼굴 결혼 한남 중국 나이 동남아 한국 보고
Topic # 1
생각 거지 우리 무조건 엄마 주제 정도 시골 우리나라 백인
Topic # 2
사람 장애인 무슨 게이 여자애 장애 자기 부모 우리 게임
Topic # 3
인간 원래 집단 인물 여성 페미 조선족 누가 인간 집단 인간 인간
Topic # 4
그냥 전라도 사람 냄새 요즘 공부 출신 저런 인간 이유
Topic # 5
남자 역시 짱깨 나라 머리 문제 여자 남자 남자 여자 그게 화장


C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [27]:
doc_topic = lda.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    topic_pr = doc_topic[n].max()
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

doc_topic_df = doc_topic_df.join(df)

for topic in range(len(doc_topic_df['Topic'].unique())):
    print('Topic #', topic, '-----------------------------')
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    print(top_pr_topics['text'].iloc[0])
    print(top_pr_topics['text'].iloc[1])
    print(top_pr_topics['text'].iloc[2], '\n')

Topic # 0 -----------------------------
이제 한국도 중국인이 모든 부동산을 싹쓸이하여 중국 됩니다
한국의 차고 넘치는 잡대학 나왔다고, 고연봉 사무직 노리는 건 진짜 노양심이지.
대학이고 나발이고 조신하게 집안일이나 하라고 해 

Topic # 1 -----------------------------
솔직히 외모도 센터에 실력도 이정도인 아이돌이 어딨냐? 잘못은 잘못이고 나같아도 안고 감
우리 엄마가 임대 아파트 사는 애들하고는 어울리지 말랬어
인정, 너무 똑똑하고 되바라진 여자는 여자로서 가치가 없음. 

Topic # 2 -----------------------------
난 뚱뚱한 사람 보면 자기 관리도 제대로 못 하는 거 같아서 한심해
뚱뚱한 사람은 자기 관리 못하는 사람이야
돈이 필요해서 우리 아들 만나는 거 알아 돈줄테니까 우리 아들이랑 그만 만나 

Topic # 3 -----------------------------
결국 남은 건 막대한 손해에 아줌마 #@인간및인간집단.인물.인물_무정보.여성#, 한 물 간 걸그룹임
근데 #@인간및인간집단.인물.인물_무정보.여성#는 고향이 어디야?
#@인간및인간집단.인물.인물_무정보.여성#이가 얼마나 모범생인데 니가 시비를 걸었겠지 

Topic # 4 -----------------------------
#@인간및인간집단.조직및단체.교육기관# 출신의 성공 비결은 오직 취집이지
지잡대 교수보다 #@인간및인간집단.조직및단체.교육기관# 학부생이 공부 더 잘할 것 같은데?
아, 제가 #@인간및인간집단.조직및단체.교육기관#를 다니는데, 제가 자리를 쓰는 게 더 효율적이지 않을까 해서요. 

Topic # 5 -----------------------------
연예인이 아닌 이상 남자가 화장하면 그게 게이지.
생리휴가 너무 불공평하지 않냐? 그게 그렇게 불편해? 우린 군대도 갔는데
군인 포상 휴가를 생리휴가랑 같다고 비교하고 있다 ㅋㅋ 저게 뭐임? 



## 7개

In [29]:
lda = LatentDirichletAllocation(n_components=7)
lda.fit(feat_vect)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic # 0
진짜 짱깨 그냥 주제 냄새 사람 중국인 중국 시골 능력
Topic # 1
장애인 우리 요즘 공부 동네 사람 보기 저런 아들 학교
Topic # 2
인간 전라도 집단 조선족 인간 집단 인간 인간 나라 인물 출신 집단 인물
Topic # 3
남자 역시 한국 게이 나이 머리 여자 남자 남자 여자 자기 누가
Topic # 4
흑인 원래 남자 무슨 문제 진짜 페미 동남아 무조건 보고
Topic # 5
결혼 생각 한남 여자애 엄마 사람 절대 여자 결혼 우리 이제
Topic # 6
남자 사람 거지 얼굴 하나 화장 어디 부모 애가 군대


C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [23]:
doc_topic = lda.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    topic_pr = doc_topic[n].max()
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

doc_topic_df = doc_topic_df.join(df)

for topic in range(len(doc_topic_df['Topic'].unique())):
    print('Topic #', topic, '-----------------------------')
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    print(top_pr_topics['text'].iloc[0])
    print(top_pr_topics['text'].iloc[1])
    print(top_pr_topics['text'].iloc[2], '\n')

Topic # 0 -----------------------------
여성들은 공산주의를 통해서 남성에게 의존할 필요없고 육아와 가사노동에 자신의 인생을 낭비할 필요도 없게된다.
돈이 필요해서 우리 아들 만나는 거 알아 돈줄테니까 우리 아들이랑 그만 만나
우리가 내는 세금으로 재난지원금 받아먹으면서 왜이렇게 불만이 많아? 

Topic # 1 -----------------------------
한국의 차고 넘치는 잡대학 나왔다고, 고연봉 사무직 노리는 건 진짜 노양심이지.
이게 진짜 남녀차별 이지.. 그래 놓고 남편 퇴근하면 어제 먹다 남은 반찬주고..
교포들은 근본이 어디냐 지 맘대로 영어 썼다 한국어 썼다 잡종같음 

Topic # 2 -----------------------------
니도 그 나이대에 별반 차이없는 것 같은데? 부모한테 가정 교육 안 받은 티좀 내지마 저능아야
나는 인종차별하는사람들이 이해가 안됐는데 오늘 흑인 만나보고 이해가 되더라
난 뚱뚱한 사람 보면 자기 관리도 제대로 못 하는 거 같아서 한심해 

Topic # 3 -----------------------------
솔직히 지네 나라에선 능력있는 사람들이긴 했겠지만 우리나라에서는 막노동이나 하겠지 뭐ㅋㅋ
뭐야 이 아저씨 대리기사 주제에 설치네?
우리나라 사람들은 아직도 일본만 보면 개거품 물어 

Topic # 4 -----------------------------
ㅋㅋ#@인간및인간집단.인물.인물_무정보.여성#씨는 여자인데 결혼하고 애 낳을 계획 있어요?그러면 채용에 쫌 곤란한데..
결국 남은 건 막대한 손해에 아줌마 #@인간및인간집단.인물.인물_무정보.여성#, 한 물 간 걸그룹임
#@인간및인간집단.조직및단체.교육기관# 출신의 성공 비결은 오직 취집이지 

Topic # 5 -----------------------------
동남아는 항상 비가 와서 사람들도 냄새가 나나?
맞아. 정말 역겨워. 사람 냄새가 아니라 음식물쓰레기냄새같아
대체 #@# 난민들을 왜 우리나라에 데

## 9개

In [20]:
lda = LatentDirichletAllocation(n_components=9)
lda.fit(feat_vect)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic # 0
진짜 그냥 역시 얼굴 흑인 사람 보고 그게 동남아 백인
Topic # 1
장애인 전라도 요즘 사람 수준 시골 자기 커피 이해 사회
Topic # 2
생각 결혼 정도 주제 나이 페미 동네 자식 사람 때문
Topic # 3
사람 우리 한국 중국 저런 진짜 중국인 아들 여성 서울
Topic # 4
남자 냄새 어디 공부 머리 여자 남자 남자 여자 조선족 애가 무슨
Topic # 5
하나 엄마 보기 누가 거기 그냥 별로 임신 정상 레즈
Topic # 6
남자 거지 인간 짱깨 나라 한남 화장 군대 집단 인간 집단
Topic # 7
게이 무슨 여자애 문제 인간 이유 집단 장애 운전 인물
Topic # 8
무조건 차별 원래 정신병 게임 이제 성격 범죄 운동 자고로


C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


## 10개

In [21]:
lda = LatentDirichletAllocation(n_components=10)
lda.fit(feat_vect)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic # 0
원래 페미 때문 지방 장애 잼민 게임 동남아 친구 인생
Topic # 1
사람 생각 중국 여성 사랑 건가 얘기 이해 소방관 동남아
Topic # 2
무슨 어디 엄마 머리 정도 여자 여자 그거 알바 담배 걱정
Topic # 3
남자 조선족 여자애 한국 남자 여자 보기 절대 이제 자식 어차피
Topic # 4
인간 게이 집단 문제 주제 인간 인간 인간 집단 인물 보고 집단 인물
Topic # 5
장애인 역시 하나 한남 냄새 나이 누가 인정 운동 흑인
Topic # 6
남자 흑인 나라 여자 남자 능력 거기 군대 범죄 완전 생각
Topic # 7
그냥 거지 전라도 사람 우리 동네 중국인 저런 자기 학교
Topic # 8
진짜 요즘 공부 무조건 부모 화장 자고로 수저 생각 흑형
Topic # 9
짱깨 결혼 얼굴 수준 아들 백인 우리 시골 여자 결혼 일본


C:\Users\user\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
